In [1]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from rasterio.mask import mask
import ee
import folium
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import io

In [2]:
# Initialize the Earth Engine module
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
# Define the administrative boundary of Pará, Brazil (need to filter grids)
Admin = ee.FeatureCollection("FAO/GAUL/2015/level1")  # Load the GAUL dataset

# Filter the collection to get the boundary of Pará, Brazil
study_area_ee = Admin.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Para'),
    ee.Filter.eq('ADM0_NAME', 'Brazil')
))

In [4]:
# Define start and end date for data search
start_date = '2024-01-01'
end_date = '2024-11-25'

In [18]:
# Function to apply cloud masking using the QA_PIXEL band
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 5).eq(0))
    return image.updateMask(cloud_mask)

In [19]:
# Function to apply scale factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [20]:
# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)


In [21]:
# Search for Landsat 8 and 9 images using GEE
landsat_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                      .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2'))
                      .filterBounds(study_area_ee)
                      .filterDate(start_date, end_date)
                      .filterMetadata('CLOUD_COVER', 'less_than', 10)
                      .map(mask_clouds)
                      .map(apply_scale_factors)
                      .map(calculate_ndvi))

In [22]:
# Get the image with the maximum NDVI value per pixel and add all bands from the corresponding image
max_ndvi = landsat_collection.qualityMosaic('NDVI').clip(study_area_ee)

In [23]:
# Define visualization parameters
true_color_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0,
    'max': 0.2,
    'region': study_area_ee.geometry().getInfo()
}


In [24]:
# Plot the true color composite using folium
map_center = [-3.7, -52.5]
folium_map = folium.Map(location=map_center, zoom_start=6)

# Convert the Earth Engine image to a URL for visualization
map_id_dict = ee.Image(max_ndvi).getMapId(true_color_params)
tile_url = map_id_dict['tile_fetcher'].url_format

# Add the layer to the folium map
folium.TileLayer(tiles=tile_url, attr='Google Earth Engine', name='Max NDVI True Color', overlay=True).add_to(folium_map)

# Add layer control
folium.LayerControl().add_to(folium_map)

# Display the map
folium_map